https://medium.com/agents-and-robots/the-bitter-truth-python-3-11-vs-cython-vs-c-performance-for-simulations-babc85cdfef5

In [29]:
import math
import random

from datetime import datetime
random.seed(datetime.now().timestamp())

WORLD_WIDTH = 2560
WORLD_HEIGHT = 1440
TIMESTEPS = 1000

In [2]:
import julia
from julia.api import Julia
# no precompilation gives faster loading times, however GPU support through CUDA does not work
jl = Julia(compiled_modules=False)
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [3]:
class PythonAgent():
    def __init__(self, x=None, y=None, world_width=0, world_height=0):
        super().__init__()

        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x if x else random.randint(0, self.world_width)
        self.y = y if y else random.randint(0, self.world_height)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # inital values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

    def update(self, food=()):
        self.age = self.age + 1

        # we can't move
        if self.vmax == 0:
            return

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x - self.target.x) ** 2 + (self.y - self.target.y) ** 2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a
            if min_dist < 100000:
                self.target = min_agent

        # initalize 'forces' to zero
        fx = 0
        fy = 0

        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05 * fx
        self.dy = self.dy + 0.05 * fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the world boundaries
        self.x = max(self.x, 0)
        self.x = min(self.x, self.world_width)
        self.y = max(self.y, 0)
        self.y = min(self.y, self.world_height)

In [4]:
from agent import Agent as CythonAgent

In [26]:
%%julia
@pydef mutable struct JuliaAgent
    function __init__(self, x=10)
        self.x = x
    end
    function __init__(self; x=nothing, y=nothing, world_width=0, world_height=0)
        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x !== nothing ? x : rand(0:world_width)
        self.y = y !== nothing ? y : rand(0:world_height)

        # initial velocity
        self.dx = 0.0
        self.dy = 0.0

        # inital values
        self.is_alive = true
        self.target = nothing
        self.age = 0
        self.energy = 0
        return
    end
    
    function update(self, food)
        update!(self, food)
    end
end

function update!(self, food::Vector)
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

<PyCall.jlwrap update!>

In [27]:
JuliaAgent = %julia JuliaAgent

In [32]:
def main(Agent):

    class Predator(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.5

    class Prey(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.0

    class Plant(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 0

            
    # open the ouput file
    # f = open('output.csv', 'w')
    # print(0, ',', 'Title', ',', 'Predator Prey Relationship / Example 02 / Cython', file=f)

    # create initial agents
    kwargs = dict(
        world_width=WORLD_WIDTH,
        world_height=WORLD_HEIGHT,
    )
    preys = [Prey(**kwargs) for i in range(10)]
    predators = [Predator(**kwargs) for i in range(10)]
    plants = [Plant(**kwargs) for i in range(100)]

    timestep = 0
    while timestep < TIMESTEPS:
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        [a.update(plants) for a in preys]
        [a.update(preys) for a in predators]

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant(**kwargs) for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
        # print("timestep: ", timestep) 

    print(len(predators), len(preys), len(plants))

In [33]:
%timeit -r 3 main(PythonAgent)

11 20 1741
11 0 1946
10 27 1785
10 0 1956
313 ms ± 115 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [34]:
%timeit main(CythonAgent)

11 35 1582
10 5 2014
10 0 2033
10 13 1925
10 0 2043
10 51 1547
10 27 1645
10 20 1903
10 7 1980
10 10 1924
10 14 1901
10 31 1736
10 15 1831
10 2 2025
10 12 1879
10 25 1789
10 28 1804
10 0 2034
10 49 1586
11 3 1918
10 9 1918
10 17 1825
10 2 1980
10 24 1867
10 0 2029
10 38 1736
10 11 1851
10 20 1846
10 31 1722
10 33 1710
10 8 1922
10 0 2032
10 34 1762
10 3 1941
10 25 1718
10 0 2028
10 12 1936
10 30 1802
10 11 1961
10 9 1931
10 0 2052
10 13 1876
10 9 1954
10 25 1821
10 22 1809
10 1 2064
10 11 1889
10 4 2032
10 45 1678
10 33 1679
10 17 1835
11 3 1935
10 1 2012
11 8 1875
10 23 1892
10 5 1954
10 22 1833
10 16 1888
10 15 1885
10 1 2007
10 31 1741
10 3 1960
11 28 1659
10 0 2055
10 8 1885
10 22 1855
10 17 1871
10 0 2028
10 6 1986
10 35 1771
10 74 1513
10 37 1691
10 16 1934
10 30 1722
10 20 1896
10 30 1804
10 28 1660
10 16 1935
10 35 1705
10 74 1463
10 35 1765
116 ms ± 8.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
main(JuliaAgent)

10 45 1683


In [35]:
main(JuliaAgent)

10 35 1721


In [24]:
%timeit main(JuliaAgent)

KeyboardInterrupt: 

# C++

```bash
g++ example.cpp -o example -std=c++11 -O3
```

In [20]:
!perf stat -r 100 -ddd ./example $TIMESTEPS

ok 0, 1304, 93
ok 0, 1304, 93
ok 0, 1306, 82
ok 0, 1306, 82
ok 0, 1306, 82
ok 0, 1302, 80
ok 0, 1302, 80
ok 0, 1300, 90
ok 0, 1300, 90
ok 26, 266, 562
ok 26, 266, 562
ok 26, 266, 562
ok 0, 1306, 86
ok 0, 1306, 86
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 1290, 73
ok 0, 1290, 73
ok 0, 1290, 73
ok 0, 1291, 101
ok 0, 1291, 101
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 1301, 80
ok 0, 1301, 80
ok 14, 578, 228
ok 14, 578, 228
ok 14, 578, 228
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 1286, 77
ok 0, 1286, 77
ok 0, 1289, 85
ok 0, 1289, 85
ok 0, 1289, 85
ok

In [4]:
%%julia
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

<PyCall.jlwrap Plant>

In [5]:
%%julia
function update!(self::Agent, food::Vector{Agent})
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

<PyCall.jlwrap update!>

In [6]:
%%julia
function main(; show=true)
    kwargs = (
        world_width = $WORLD_WIDTH,
        world_height = $WORLD_HEIGHT,
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < $TIMESTEPS
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    if show
        println(length(predators), length(preys), length(plants))
    end
end

<PyCall.jlwrap main>

In [19]:
%julia using BenchmarkTools
%julia @btime main(show=false)

  419.282 ms (47250 allocations: 105.58 MiB)


In [17]:
%julia display(@benchmark main(show=false))

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  151.085 ms … 668.042 ms  ┊ GC (min … max): 0.00% … 1.45%
 Time  (median):     492.457 ms               ┊ GC (median):    1.91%
 Time  (mean ± σ):   468.291 ms ± 173.361 ms  ┊ GC (mean ± σ):  4.91% ± 7.06%

  █                                ▁▁ ▁   ▁       ▁▁  ▁   ▁   ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁█▁▁▁█▁▁▁▁▁▁▁██▁▁█▁▁▁█▁▁▁█ ▁
  151 ms           Histogram: frequency by time          668 ms <

 Memory estimate: 4.35 MiB, allocs estimate: 30296.


In [3]:
# import os
# os.environ["JULIA_PYTHONCALL_EXE"] = "@PyCall"

In [2]:
from juliacall import Main as jl

In [9]:
jl.seval("""begin
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

function update!(self::Agent, food::Vector{Agent})
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end
end
""")

update! (generic function with 1 method)

In [10]:
jl.seval("""
function main(; show=true)
    kwargs = (
        world_width = {WORLD_WIDTH},
        world_height = {WORLD_HEIGHT},
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < {TIMESTEPS}
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    if show
        println(length(predators), length(preys), length(plants))
    end
end
""".format(WORLD_HEIGHT=WORLD_HEIGHT, WORLD_WIDTH=WORLD_WIDTH, TIMESTEPS=TIMESTEPS))

main (generic function with 1 method)

In [11]:
jl.seval("using BenchmarkTools")

In [13]:
jl.seval("@benchmark main(show=false)")

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  167.633 ms … 845.906 ms  ┊ GC (min … max): 0.00% … 23.42%
 Time  (median):     456.908 ms               ┊ GC (median):    2.08%
 Time  (mean ± σ):   462.981 ms ± 185.949 ms  ┊ GC (mean ± σ):  5.56% ±  6.64%

  ▁▁                   ▁ █  ▁    ▁█     ▁                     ▁  
  ██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁█▁▁▁▁██▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  168 ms           Histogram: frequency by time          846 ms <

 Memory estimate: 4.47 MiB, allocs estimate: 30293.